# Supplementary File: Analysis Code for Local and Global Heritability and Genetic Correlation Estimation

This Jupyter Notebook provides the code used for the analyses in our study, focusing on the genetic architecture of the HLA region across ancestries. We use the Total Cholesterol (TC) phenotype in the African American (AFR) population as a running example, with a particular emphasis on the HLA region for local analyses.

## 1. Local Heritability Estimation with KGGSEE EHE

This script calculates local heritability using the KGGSEE Effective Heritability Estimator (EHE). It processes Genome-Wide Association Study (GWAS) summary statistics chromosome by chromosome and region by region, with special attention to the HLA region.

In [ ]:
%%bash

# Define directories
# Path to the directory where output files will be saved
OUT_DIR=/path/to/output
# Path to the directory containing GWAS summary statistics
SST_DIR=/path/to/summary_statistics
# Path to the directory containing reference panel files (e.g., 1000 Genomes)
REF_DIR=/path/to/reference_panels
# Path to the directory where the KGGSEE tool is located
TOOL_DIR=/path/to/tools

# Set parameters for our example
POPULATION=AFR
REF_POPULATION=afr  # 1000 Genomes African reference population identifier
PHENO=TC            # Total Cholesterol phenotype
A2=A2               # Column name for the non-effect allele in the summary statistics

# Process chromosome 6 for the HLA region
chr=6
# Define HLA region (region number will depend on your specific genomic segmentation, this is an example)
# This numeric identifier corresponds to the HLA region within the chromosome's defined segments.
region_hla=13

# Run KGGSEE for the HLA region
# Ensure kggsee.jar is executable and paths are correctly set.
java -Xmx10g -jar ${TOOL_DIR}/kggsee.jar \
 --gene-herit \
 --out ${OUT_DIR}/${PHENO}/${POPULATION}/chr${chr}/${PHENO}_${POPULATION}_chr${chr}_region${region_hla} \
 --vcf-ref ${REF_DIR}/${POPULATION}/1kg.phase3.v5.shapeit2.${REF_POPULATION}.hg19.chr${chr}.vcf.gz \
 --sum-file ${SST_DIR}/${POPULATION}/${PHENO}/MVP.${PHENO}.${POPULATION}.chr${chr}.txt \
 --nmiss-col SampleSize \
 --chrom-col Chromosome \
 --pos-col Position \
 --p-col PValue \
 --a1-col EA \
 --a2-col ${A2} \
 --freq-a1-col EAF \
 --beta-col Effect \
 --beta-type 0 \
 --se-col SE \
 --neargene 5000 \
 --buildver hg19 \
 --regions-bed /path/to/your/regions.bed \
 --ld-block-max-r2 0.15 \
 --adjust-gc 1

The `regions.bed` file should contain the coordinates of your regions of interest. Example content for a BED file focusing on the HLA region:
```
chr6    28477797    33448354    HLA
```

## 2. Local Genetic Correlation Estimation with LAVA

This R script calculates local genetic correlations within the HLA region using LAVA. It focuses on pairwise correlations involving TC in the AFR population.

In [ ]:
// Ensure you have an R kernel (e.g., IRkernel) installed and selected for this cell.
// Or run this code in an R environment.

# Load the LAVA library
# Ensure LAVA is installed in your R environment: install.packages("LAVA")
library(LAVA)

# Process input for the African American population
input = process.input(
    input.info.file="input.info.txt",       # input info file
    sample.overlap.file=NULL,               # sample overlap file (can be set to NULL if there is no overlap)
    ref.prefix="g1000_afr",                 # reference genotype data prefix
    phenos=c("TC", "BMI", "CIHS", "HDL", "IDE", "LDL", "MDD", "MaxAlc", "SITB", "TG") # subset of phenotypes listed in the input info file that we want to process
)

# Read in locus info file (e.g., "hla.locus")
# The locus file (e.g., "hla.locus") should define loci with columns: LOC_ID CHR START STOP
# Example line for "hla.locus":
# HLA 6 28477797 33448354
loci = read.loci("hla.locus")      # Read the locus definition file.
                                     # To inspect, use head(loci). For format details: ?read.loci

# Create a locus object for the first locus (e.g., HLA) to prepare it for analysis
hla_locus = process.locus(loci[1,], input) # Process the first locus from the file.

# Perform univariate analysis for all specified phenotypes in the HLA locus
# This step estimates heritability for each phenotype within the defined locus.
univ_results = run.univ(hla_locus)

# Calculate bivariate local genetic correlations between pairs of phenotypes within the HLA locus
bivar_results = run.bivar(hla_locus, param.lim = 1.30)

## 3. Global Heritability Estimation with LDAK

This script calculates global (genome-wide) heritability for TC in the AFR population using LDAK.

In [ ]:
%%bash

# Define parameters
POP="AFR"
PHENO="TC"
# Path to the directory containing GWAS summary statistics
DATA_DIR="/path/to/summary_statistics_for_ldak"
# Path to the directory where LDAK output files will be saved
OUT_DIR="/path/to/output_ldak_heritability"

# Ensure the output directory exists
mkdir -p ${OUT_DIR}/${PHENO}_${POP}

# Print information
echo "Calculating global heritability for ${PHENO} in ${POP} population..."

# Run LDAK to calculate SNP heritability
# Ensure ldak5.2.linux (or your LDAK version) is executable and in your PATH or specify the full path.
ldak5.2.linux \
    --sum-hers ${OUT_DIR}/${PHENO}_${POP}/${PHENO}_${POP}_snpher2 \
    --summary ${DATA_DIR}/${PHENO}_${POP}/${PHENO}_${POP}.txt \
    --tagfile ${DATA_DIR}/bld.ldak.hapmap.${POP}.tagging

Notes for LDAK parameters:

* `--summary`: GWAS summary statistics file for the phenotype. It typically requires columns like SNP identifier, A1, A2, N (sample size), and Z-score or Beta/SE.
* `--tagfile`: Pre-computed LDAK tagging file for the population. This file captures LD information and is crucial for LDAK's heritability model.
* `--sum-hers`: Performs heritability estimation from summary statistics.

## 4. Global Genetic Correlation Estimation with LDAK

This script calculates global genetic correlations between TC and other phenotypes in the AFR population using LDAK.

In [ ]:
%%bash

# Define directories and parameters
# Path to the directory containing GWAS summary statistics for all phenotypes
SUM_DIR="/path/to/summary_statistics_for_ldak_correlations"
# Path to the directory where LDAK global correlation output files will be saved
OUT_DIR="/path/to/output_ldak_global_correlations"
POP="AFR"
POP_REF="afr" # Reference population code for 1000 Genomes (used in tagging file name convention)

# Ensure the output directory exists
mkdir -p ${OUT_DIR}/LDAK_${POP}

# Define phenotypes list with TC as our focus phenotype
PHENOS=("TC" "BMI" "CIHS" "HDL" "LDL" "IDE" "SITB" "TG" "MaxAlc" "MDD")

# Calculate genetic correlation between TC and each other phenotype
for PHENO2 in "${PHENOS[@]}"; do
    if [ "$PHENO2" != "TC" ]; then
        echo "Calculating global genetic correlation between TC and ${PHENO2} in ${POP} population..."
        
        # Ensure ldak5.2.linux (or your LDAK version) is executable and in your PATH or specify the full path.
        ldak5.2.linux \
            --sum-cors ${OUT_DIR}/LDAK_${POP}/TC_${PHENO2}_${POP}_gencor \
            --summary ${SUM_DIR}/TC_${POP}.txt \
            --summary2 ${SUM_DIR}/${PHENO2}_${POP}.txt \
            --tagfile ${SUM_DIR}/ldak.thin.hapmap.${POP_REF}.tagging
    fi
done

Notes for LDAK parameters in loop:

* `--tagfile`: Pre-computed LDAK tagging file (thinned version often used for correlations).

## Tool Documentation

For detailed information on the data format requirements and further options for each tool, please refer to their respective documentation:
* **KGGSEE EHE:** [https://kggsee.readthedocs.io/en/latest/detailed_document.html#ehe-gene-based-heritability-estimation](https://kggsee.readthedocs.io/en/latest/detailed_document.html#ehe-gene-based-heritability-estimation)
* **LAVA:** [https://github.com/josefin-werme/LAVA](https://github.com/josefin-werme/LAVA)
* **LDAK:** [https://dougspeed.com/ldak/](https://dougspeed.com/ldak/)